In [1]:
import pandas as pd
import sqlite3
db_file = 'sqlite_db_pythonsqlite.db'
conn = sqlite3.connect(db_file)

### QUESTIONS

#### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [4]:
query1 = """
SELECT name FROM Facilities
WHERE membercost > 0.0;
"""
pd.read_sql_query(query1, conn)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


##### Q2: How many facilities do not charge a fee to members?

In [6]:
query2 = """
SELECT COUNT(name) AS No_Charge 
FROM Facilities
WHERE membercost = 0.0
"""
pd.read_sql_query(query2, conn)

,No_Charge
0,4


##### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [7]:
query3 = """
SELECT  `facid` ,  `name` ,  `membercost` ,  `monthlymaintenance` 
FROM Facilities
WHERE  `membercost` > 0.0
AND  `membercost` < (`monthlymaintenance` -  `monthlymaintenance` * 0.2) 
"""
pd.read_sql_query(query3, conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


##### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [8]:
query4= """
SELECT * FROM Facilities
WHERE `facid` IN (1,5);
"""
pd.read_sql_query(query4,conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


##### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [9]:
query5= """
SELECT `name`, `monthlymaintenance`, 
CASE WHEN `monthlymaintenance`> 100 THEN 'expensive'
ELSE 'cheap' END AS category
FROM Facilities;
"""
pd.read_sql_query(query5,conn)

,name,monthlymaintenance,category
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


##### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [10]:
query6= """
SELECT * FROM `Members` 
WHERE `joindate` = (SELECT MAX(`joindate`) FROM Members);
"""
pd.read_sql_query(query6,conn)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,37,Smith,Darren,"3 Funktown, Denzington, Boston",66796,(822) 577-3541,,2012-09-26 18:08:45


##### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
query7= """
SELECT sub.court, CONCAT(sub.firstname, ' ', sub.lastname) AS name
FROM
(SELECT F.name AS court, M.firstname AS firstname, M.surname AS lastname
FROM Bookings B
INNER JOIN Facilities F ON F.facid=B.facid
AND F.name LIKE 'Tennis Court%'
INNER JOIN Members M ON B.memid = M.memid
) AS sub
GROUP BY sub.court, sub.firstname,sub.lastname
ORDER BY name;
"""
pd.read_sql_query(query7,conn)

##### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [ ]:
query8= """
SELECT Facilities.name AS facility, CONCAT(Members.firstname, ' ', Members.surname) AS name, 
CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
ELSE Facilities.membercost * Bookings.slots 
END AS cost

FROM Bookings

Inner JOIN Members ON Bookings.memid=Members.memid

Inner JOIN Facilities ON Bookings.facid=Facilities.facid
AND Bookings.starttime LIKE '2012-09-14%'
AND (((Bookings.memid =0) AND (Facilities.guestcost * Bookings.slots >30))
OR ((Bookings.memid !=0) AND (Facilities.membercost * Bookings.slots >30)))

ORDER BY cost;
"""
pd.read_sql_query(query8,conn)

###### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
query9= """
SELECT * FROM 
(SELECT Facilities.name AS facility, CONCAT(Members.firstname, ' ', Members.surname) AS name, 

CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
ELSE Facilities.membercost * Bookings.slots 
END AS cost

FROM Bookings

Inner JOIN Members ON Bookings.memid=Members.memid

Inner JOIN Facilities ON Bookings.facid=Facilities.facid
AND Bookings.starttime LIKE '2012-09-14%') sub

WHERE sub.cost >30

ORDER BY sub.cost;
"""
pd.read_sql_query(query9,conn)